In [10]:
import torch

n = 5
a = torch.ones(n, n)
pe_list = [a, 2*a, -1*a]
pe = torch.stack(pe_list, dim=-1)
pe.shape

torch.Size([5, 5, 3])

In [11]:
pe[:, :, 0]

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [12]:
pe

tensor([[[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]]])

In [13]:
from torch_sparse import SparseTensor
abs_pe = pe.diagonal().transpose(0, 1) # n x k
rel_pe = SparseTensor.from_dense(pe, has_value=True)
rel_pe_row, rel_pe_col, rel_pe_val = rel_pe.coo()
rel_pe_idx = torch.stack([rel_pe_row, rel_pe_col], dim=0)
rel_pe_idx

tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
         4],
        [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
         4]])

In [14]:
pe

tensor([[[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]],

        [[ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.],
         [ 1.,  2., -1.]]])

In [15]:
pe.abs().sum([i for i in range(2, pe.dim())])

tensor([[4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.],
        [4., 4., 4., 4., 4.]])

In [6]:
rel_pe_idx.shape, rel_pe_val.shape

(torch.Size([2, 25]), torch.Size([25, 3]))

In [5]:
from torch_scatter import scatter

def full_edge_index(edge_index, batch=None):
    """
    Retunr the Full batched sparse adjacency matrices given by edge indices.
    Returns batched sparse adjacency matrices with exactly those edges that
    are not in the input `edge_index` while ignoring self-loops.
    Implementation inspired by `torch_geometric.utils.to_dense_adj`
    Args:
        edge_index: The edge indices.
        batch: Batch vector, which assigns each node to a specific example.
    Returns:
        Complementary edge index.
    """

    if batch is None:
        batch = edge_index.new_zeros(edge_index.max().item() + 1)

    batch_size = batch.max().item() + 1
    one = batch.new_ones(batch.size(0))
    num_nodes = scatter(one, batch,
                        dim=0, dim_size=batch_size, reduce='add')
    cum_nodes = torch.cat([batch.new_zeros(1), num_nodes.cumsum(dim=0)])

    negative_index_list = []
    for i in range(batch_size):
        n = num_nodes[i].item()
        size = [n, n]
        adj = torch.ones(size, dtype=torch.short,
                         device=edge_index.device)

        adj = adj.view(size)
        _edge_index = adj.nonzero(as_tuple=False).t().contiguous()
        # _edge_index, _ = remove_self_loops(_edge_index)
        negative_index_list.append(_edge_index + cum_nodes[i])

    edge_index_full = torch.cat(negative_index_list, dim=1).contiguous()
    return edge_index_full

edge_index = torch.tensor([[0, 1, 1, 2],
                            [1, 0, 2, 1]], dtype=torch.long)

batch = edge_index.new_zeros(edge_index.max().item() + 1)
one = batch.new_ones(batch.size(0))
batch_size = batch.max().item() + 1
num_nodes = scatter(one, batch,
                    dim=0, dim_size=batch_size, reduce='add')
cum_nodes = torch.cat([batch.new_zeros(1), num_nodes.cumsum(dim=0)])
cum_nodes

tensor([0, 3])

In [1]:
import torch

N = 10
A = 3
B = 4

a = torch.randn(N, A)
b = torch.randn(N, B)

a + b

/home/a/anaconda3/envs/grit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1

In [ ]:
from torch_sparse import SparseTensor
